In [36]:
'''
import general python libraries
'''
import numpy as np
from collections import Counter
import math

In [37]:
'''
import project specific libraries
'''
import StationV3 as Station
import Prices

In [58]:
'''
------------------------------------------------------------------------------------------------------------
set when the HGV will arrive and the number of electrolysers at the station
'''
dispenser_time_demand = [10,20,30,40]
# the number below is set at 1 initially but the optimisation will increase this until the conditions are satisfied
number_electorlysers = 1

# --------------------
# making sure the while loop can start
unsatisfaction = np.array([1])
# --------------------
'''
------------------------------------------------------------------------------------------------------------
'''



# ----------------------------------------------------------------------------------------------------------
# running a while loop to get the minimum number of canisters and the minimum number of electrolysers
# ----------------------------------------------------------------------------------------------------------
while np.sum(unsatisfaction) > 0:
    '''
    ------------------------------------------------------------------------------------------------------------
    work out the minimum number of canisters to satisfy the demand at the end of the half hour period
    '''
    counter = Counter(dispenser_time_demand)
    temp = np.zeros(49)
    for n in range(49):
        temp[n] = counter[n]
    temp2 = np.argsort(temp)[-1]
    maximum = counter[temp2]
    min_number_canisters = math.ceil((maximum*32.09)/9.5)
    number_canisters = min_number_canisters
    '''
    -------------------------------------------------------------------------------------------------------------
    '''



    '''
    -------------------------------------------------------------------------------------------------------------
    work out the minimum number of canisters to satisfy the constraint that the station needs to be able to charge
    and discharge at the same time
    '''
    spare_capacity = number_canisters*9.5-maximum*32.09
    charge_time = (spare_capacity)/(0.0123*number_electorlysers)
    discharge_time = 9.5/0.0382
    while discharge_time > charge_time:
        number_canisters += 1
        spare_capacity = number_canisters*9.5-maximum*32.09
        charge_time = (spare_capacity)/(0.0123*number_electorlysers)
        discharge_time = 9.5/0.0382
    '''
    -------------------------------------------------------------------------------------------------------------
    '''






    '''
    -------------------------------------------------------------------------------------------------------------
    set up the station with its parameters
    '''

    electrolyser_capacity = 22.125*number_electorlysers
    station_capacity = 0
    storage_capacity = 9.5*number_canisters

    station1 = Station.Station(electrolyser_capacity,station_capacity,storage_capacity)
    '''
    -------------------------------------------------------------------------------------------------------------
    '''



    '''
    -------------------------------------------------------------------------------------------------------------
    run the system for 2020 price data
    '''
    total_cost = np.zeros(365)
    unsatisfaction = np.zeros(365)

    for n in range(365):

        day = n+1
        elec_price = Prices.elec_prices_data(day)

        starting_storage_level = 0

        demand, level, unsatisfied = station1.get_optimised_electrolyser_demand(elec_price, dispenser_time_demand, starting_storage_level)

        # convert from demand in kg to kWh
        demand = demand*49.465
        # convert electricity prices into pounds
        elec_price = elec_price/100
        # find the cost of the refill
        cost = np.dot(demand,elec_price)

        total_cost[n] = cost
        unsatisfaction[n] = unsatisfied
    '''
    -------------------------------------------------------------------------------------------------------------
    '''




    '''
    -------------------------------------------------------------------------------------------------------------
    work out the average refill cost
    '''
    avg_refill_cost = (np.average(total_cost))/(len(dispenser_time_demand))
    '''
    -------------------------------------------------------------------------------------------------------------
    '''




    '''
    -------------------------------------------------------------------------------------------------------------
    if any HGV's were unsatisfied then increment the number of electrolysers and run the simulation again
    '''
    if np.sum(unsatisfaction) > 0:
        number_electorlysers += 1
    '''
    -------------------------------------------------------------------------------------------------------------
    '''
# ----------------------------------------------------------------------------------------------------------
# running a while loop to get the minimum number of canisters and the minimum number of electrolysers
# ----------------------------------------------------------------------------------------------------------

print('For HGV arrival times = '+str(dispenser_time_demand))
print('')
print('Number of Electrolysers needed = '+str(number_electorlysers))
print('Number of Canisters needed = '+str(number_canisters))
print('Average Refill Cost = '+str(avg_refill_cost))


For HGV arrival times = [10, 20, 30, 40]

Number of Electrolysers needed = 1
Number of Canisters needed = 4
Average Refill Cost = 93.90893220450181
